In [ ]:
!pip install langchain
!pip install openai
# will help us to serialize the database and stored on a file system
!pip install pickle
!pip install python-dotenv
# vector store which allows us to store vectors in database
!pip install faiss-cpu

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
API_KEY = os.environ.get("API_KEY")

### Loaders

To use data with an LLM, documents must first be loaded into a vector database. The first step is to load them into memoryn via loader.


In [ ]:
from langchain.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader('./FAQ', glob = "**/*.txt", loader_cls = TextLoader, show_progress = True)

docs = loader.load()

### Text Splitter

Texts are not loaded 1:1 into the database, but in pieces, so called "chunks". You can define the chunk size and the overlap between the chunks.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 100,
)

documents = text_splitter.split_documents(docs)
documents[0]

### Embeddings

Texts are not stored as text in the database, but as vector representations. Embeddings are a type of word representation that represents the semantic meaning of words in a vector space.

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key = API_KEY)

### Loading Vectors into VectorDB (FAISS)

As created by OpenAIEmbeddings vectors can now be stored in the database. The DB can be stored as .pkl file

In [ ]:
from langchain.vectorstores.faiss import FAISS
import pickle

# from_documents is used to create vectorstore and this will store the documents and embeddings you will always know which embedding belongs to which text
vectorstore = FAISS.from_documents(documents, embeddings)

# we serialize the vectorstore and dump it to the file system since we can load it later (don't make request always to the API)
with open("vectorstore.pkl", "wb") as f:
    pickle.dump(vectorstore, f)

### Loading the database

Before using the database, it must of course be loaded again.

In [ ]:
with open("vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f)

### Prompts

With an LLM you have the possibility to give it an identity before a conversation or to define how question and answer should look like.

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """You are a helpful assistant for our restaurant.

{context}

Question: {question}
Answer here:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

### Chains
Question and answering with a vector database


With chain classes you can easily influence the behaviour of the LLM

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

chain_type_kwargs = {"prompt": PROMPT}

llm = OpenAI(openai_api_key = API_KEY)

# as_retriever: allow retrieving documents easier
qa = RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff", retriever = vectorstore.as_retriever(), chain_type_kwargs= chain_type_kwargs)

query = "When does the restaurant open?"
qa.run(query)

### Memory

In the example just shown, each request stands alone. A great strength of an LLM, however, is that it can take the entire chat history into account when responding. For this, however, a chat history must be built up from the different questions and answers. With different memory classes this is very easy in LangChain

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, output_key='answer')

### Use Memory in Chains

The memory class can now easily be used in a chain. This is recognizable, for example, by the fact that when one speaks of "it", the bot understands the rabbit in this context.

In [ ]:
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(
    llm = OpenAI(model_name = "gpt-3.5-turbo", temperature=0.7),
    memory = memory,
    retriever = vectorstore.as_retriever(),
    combine_docs_chain_kwargs = {'prompt': PROMPT}
)

query = "Do you offer vegan food?"
qa({"question": query})
qa({"question": "How much does it cost"})